In [56]:
# Pricing Options using Fast Fourier Transform (FFT) with real-life stock data
# This notebook calculates option prices using mathematical models and compares them with real-world data.

# Importing Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# Configure pandas for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Display settings for matplotlib
#plt.style.use('seaborn')


In [ ]:
# Fetch real-life stock data from Yahoo Finance
def fetch_stock_data(ticker, start_date, end_date):
    """
    Fetch stock data using yfinance and calculate additional metrics.
    
    Parameters:
        ticker (str): Stock ticker symbol (e.g., "AAPL" for Apple).
        start_date (str): Start date for the data (YYYY-MM-DD).
        end_date (str): End date for the data (YYYY-MM-DD).

    Returns:
        pd.DataFrame: DataFrame containing stock price and daily return.
    """
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]
    stock_data['Daily Return'] = stock_data['Close'].pct_change()
    stock_data.dropna(inplace=True)
    return stock_data

# Example: Fetch data for Apple (AAPL)
ticker = "AAPL"
start_date = "2023-01-01"
end_date = "2023-12-31"

stock_data = fetch_stock_data(ticker, start_date, end_date)

# Display the stock data
print(f"Stock data for {ticker}:")
stock_data.tail(10)


[*********************100%***********************]  1 of 1 completed

Stock data for AAPL:


Price,Open,High,Low,Close,Volume,Daily Return
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,
Date,,,,,,
2023-12-15,196.57,197.43,196.04,196.61,128256700,-0.00
2023-12-18,195.13,195.67,193.44,194.93,55751900,-0.01
2023-12-19,195.20,195.99,194.94,195.98,40714100,0.01
2023-12-20,195.94,196.72,193.88,193.88,52242800,-0.01
2023-12-21,195.14,196.12,192.56,193.73,46482500,-0.00
2023-12-22,194.23,194.46,192.03,192.66,37122800,-0.01
2023-12-26,192.67,192.94,191.89,192.11,28919300,-0.00
2023-12-27,191.55,192.56,190.16,192.21,48087700,0.00


In [58]:
# General Parameters for Option Pricing
S0 = float(stock_data['Close'].iloc[-1])  # Convert to scalar (float)
K = S0 * 1.05                            # Strike price (5% above current stock price)
k = np.log(K)                            # Log strike price
r = 0.03                                 # Risk-free interest rate
q = 0.01                                 # Dividend yield
T = 0.5                                  # Time to maturity (6 months)

# FFT Parameters
n = 12                                   # Exponent for 2^n
N = 2 ** n                               # Number of steps
eta = 0.25                               # Integration step size
alpha = 1.0                              # Damping factor
lambda_ = (2 * np.pi / N) / eta          # Log-strike step size
beta = np.log(K)                         # Log-strike value

# Display Parameters
print(f"Option Pricing Parameters for {ticker}:")
pd.DataFrame({
    'Parameter': ['Initial Stock Price (S0)', 'Strike Price (K)', 'Risk-free Rate (r)', 
                  'Dividend Yield (q)', 'Time to Maturity (T)', 'FFT Steps (N)', 
                  'Integration Step Size (eta)', 'Damping Factor (alpha)'],
    'Value': [S0, K, r, q, T, N, eta, alpha]
})




Option Pricing Parameters for AAPL:


C:\Users\awadh\AppData\Local\Temp\ipykernel_5568\891540450.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S0 = float(stock_data['Close'].iloc[-1])  # Convert to scalar (float)


,Parameter,Value
0,Initial Stock Price (S0),191.59
1,Strike Price (K),201.17
2,Risk-free Rate (r),0.03
3,Dividend Yield (q),0.01
4,Time to Maturity (T),0.50
5,FFT Steps (N),4096.00
6,Integration Step Size (eta),0.25
7,Damping Factor (alpha),1.00


In [59]:
# Explanation of Black-Scholes Model
# The Black-Scholes model assumes stock prices follow a geometric Brownian motion:
# S(t) = S(0) * exp((r - q - 0.5 * σ^2) * T + σ * W(t))
# Where:
# - S(t): Stock price at time t.
# - σ: Volatility of the stock (standard deviation of log returns).
# - W(t): Wiener process (random component).
# - T: Time to maturity.

# Function to Compute Characteristic Function for FFT
def compute_characteristic_function(u, params, S0, r, q, T, model):
    """
    Calculate the characteristic function for different option pricing models.

    Parameters:
        u (array): Complex inputs for the Fourier transform.
        params (list): Model-specific parameters.
        S0 (float): Current stock price.
        r (float): Risk-free interest rate.
        q (float): Dividend yield.
        T (float): Time to maturity.
        model (str): Option pricing model ("BS" for Black-Scholes).

    Returns:
        np.ndarray: Values of the characteristic function.
    """
    if model == 'BS':  # Black-Scholes Model
        volatility = params[0]
        drift = np.log(S0) + (r - q - 0.5 * volatility**2) * T
        diffusion = volatility * np.sqrt(T)
        phi = np.exp(1j * drift * u - 0.5 * (diffusion * u)**2)
    return phi


In [60]:
# Function to Compute Option Prices using FFT
def calculate_fft_values(params, S0, K, r, q, T, alpha, eta, n, model):
    """
    Calculate option prices using FFT.

    Parameters:
        params (list): Model-specific parameters.
        S0 (float): Current stock price.
        K (float): Strike price.
        r (float): Risk-free interest rate.
        q (float): Dividend yield.
        T (float): Time to maturity.
        alpha (float): Damping factor.
        eta (float): Integration step size.
        n (int): FFT exponent (2^n points).
        model (str): Option pricing model.

    Returns:
        tuple: Log-strike values and option prices.
    """
    N = 2 ** n
    delta = (2 * np.pi / N) / eta
    beta = np.log(K)
    
    # Discount Factor
    discount_factor = np.exp(-r * T)
    
    # Integration Range
    nuJ = np.arange(N) * eta
    psi_nuJ = compute_characteristic_function(nuJ - (alpha + 1) * 1j, params, S0, r, q, T, model) / (
        (alpha + 1j * nuJ) * (alpha + 1 + 1j * nuJ)
    )
    
    # FFT
    km_values = beta + delta * np.arange(N)
    w_values = eta * np.ones(N)
    w_values[0] = eta / 2  # Adjust first weight
    x_values = np.exp(-1j * beta * nuJ) * discount_factor * psi_nuJ * w_values
    y_values = np.fft.fft(x_values)
    cT_km_values = (np.exp(-alpha * km_values) / np.pi) * np.real(y_values)
    
    return km_values, cT_km_values


In [61]:
# Fetch Option Chain using yfinance
def fetch_option_chain(ticker, expiration_date):
    """
    Fetch option chain for a given ticker and expiration date.

    Parameters:
        ticker (str): Stock ticker symbol.
        expiration_date (str): Expiration date (YYYY-MM-DD).

    Returns:
        pd.DataFrame: Option chain data.
    """
    stock = yf.Ticker(ticker)
    option_chain = stock.option_chain(expiration_date)
    calls = option_chain.calls
    return calls

# Example: Fetch option chain for AAPL
expiration_date = "2025-01-03"  # Example expiration date
option_chain = fetch_option_chain(ticker, expiration_date)

# Display Option Chain
option_chain[['strike', 'lastPrice', 'impliedVolatility', 'volume']].head()


,strike,lastPrice,impliedVolatility,volume
0,100.00,153.25,0.00,1.00
1,130.00,124.19,0.00,NaN
2,135.00,118.25,0.00,1.00
3,140.00,114.45,0.00,1.00
4,145.00,109.60,0.00,1.00


In [62]:
# Plot Comparison of FFT Prices and Real Option Prices
def plot_comparison(strikes, fft_prices, real_prices):
    """
    Compare FFT option prices with real-world option prices.

    Parameters:
        strikes (array): Strike prices.
        fft_prices (array): Prices from FFT computation.
        real_prices (pd.Series): Real-world option prices.
    """
    plt.figure(figsize=(12, 6))
    plt.plot(strikes, fft_prices, label="FFT Option Prices", color="blue")
    plt.scatter(real_prices.index, real_prices.values, color="red", label="Real Option Prices")
    plt.axvline(x=K, color="green", linestyle="--", label="Strike Price (K)")
    plt.title(f"Comparison of FFT Option Prices and Real Option Prices for {ticker}")
    plt.xlabel("Strike Price")
    plt.ylabel("Option Price")
    plt.legend()
    plt.grid(True)
    plt.show()


# Extract Real Prices
real_prices = option_chain.set_index('strike')['lastPrice']
fft_strikes = np.exp(km_values)
fft_prices = cT_km_values

plot_comparison(fft_strikes, fft_prices, real_prices)


NameError: name 'cT_km_values' is not defined